In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
import aesara.tensor as at
import arviz as az
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from scipy import stats
import warnings

from deepemogp import feature_extractor
from deepemogp.signal import physio as physio
from deepemogp import datasets as datasets
from deepemogp.signal import behavior as behavior

In [2]:
warnings.simplefilter(action="ignore", category=FutureWarning)
scaler = StandardScaler()

In [3]:
show = False
# definition of the feature extractors to be used later
f2 = feature_extractor.FE('wavelet', window=(2, 1))
f3 = feature_extractor.FE('mean', window=(1,0))

# definition of the physiological signals to be extracted
eda_ = physio.EDA(f3)
hr_ = physio.HR(f3)
pupil_ = behavior.PUPIL(f3)


# extraction of the desired data from the dataset
d = datasets.FEAR(signals={hr_,pupil_,eda_}, subjects={'45'})

for s in d.signals:
    # preprocess ...
    if s.name =='EDA':
        s.preprocess(show=show,new_fps=500)
        s.feature_ext.extract_feat(s,show=show)
    else:
        if s.name == 'HR':
            list_hr_test = s.raw[0]['data']
            s.preprocess(show=show, useneurokit=True)
            s.feature_ext.extract_feat(s,show=show)

        else:
            s.feature_ext.extract_feat_without_preprocess(s, show=show)

for sig in d.signals:
    if sig.name=='EDA':
        eda_data = sig.features
    if sig.name=='HR':
        hr_data = sig.features
    if sig.name=='PUPIL':
        pupil_data = sig.features

>> Loading PUPIL for subject 45 and all sessions from dataset FEAR
>> Loading HR for subject 45 and all sessions from dataset FEAR
>> Loading EDA for subject 45 and all sessions from dataset FEAR
>> Extracting mean features from PUPIL signal, adopting (1, 0) window ...
>> Processing HR ... using neurokit


/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis

>> Extracting mean features from HR signal, adopting (1, 0) window ...
>> Processing EDA ...
>> Extracting mean features from EDA signal, adopting (1, 0) window ...


In [4]:
TRIAL = 160
hr = np.array(hr_data)
hr = hr.reshape((TRIAL, int(hr.shape[0] / TRIAL * hr.shape[1])))
pupil = np.array(pupil_data)
pupil = pupil.reshape((TRIAL, int(pupil.shape[0] / TRIAL * pupil.shape[1])))
eda = np.array(eda_data)
eda = eda.reshape((TRIAL, int(eda.shape[0] / TRIAL * eda.shape[1])))

In [6]:
trial_i = 113
index = np.arange(6)
hr_trial = hr[trial_i]
eda_trial = eda[trial_i]

coords_ = {'time':index, 'hr_': hr_trial, 'eda_':eda_trial}
coords_

{'time': array([0, 1, 2, 3, 4, 5]),
 'hr_': array([141.8857144 , 158.83336858, 138.11937399, 117.17355808,
        128.07816207, 149.82723018]),
 'eda_': array([0.0937584 , 0.09504176, 0.25697106, 0.29961316, 0.22274746,
        0.35046387])}

In [7]:
with pm.Model(coords=coords_) as rolling:
    sigma_alpha = pm.Exponential("sigma_alpha", 50.0)
    sigma_beta = pm.Exponential("sigma_beta", 50.0)

    alpha = pm.GaussianRandomWalk(
        "alpha", sigma=sigma_alpha, init_dist=pm.Normal.dist(0, 10), dims="time"
    )
    beta = pm.GaussianRandomWalk(
        "beta", sigma=sigma_beta, init_dist=pm.Normal.dist(0, 10), dims="time"
    )
    regression = alpha + beta * coords_['hr_']

    # Assume prices are Normally distributed, the mean comes from the regression.
    sd = pm.HalfNormal("sd", sigma=0.1)
    likelihood = pm.Normal("y", mu=regression, sigma=sd, observed=coords_['eda_'])

In [8]:
graph_rolling = pm.model_to_graphviz(rolling)
graph_rolling.view('rolling_regr')

'rolling_regr.pdf'

In [9]:
with rolling:
    trace = pm.sample(tune=2000, target_accept=0.9)

/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/aeppl/joint_logprob.py:130: UserWarning: Found a random variable that was neither among the observations nor the conditioned variables: [normal_rv{0, (0, 0), floatX, False}.0, normal_rv{0, (0, 0), floatX, False}.out]
  warnings.warn(
/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/aeppl/joint_logprob.py:130: UserWarning: Found a random variable that was neither among the observations nor the conditioned variables: [normal_rv{0, (0, 0), floatX, False}.0, normal_rv{0, (0, 0), floatX, False}.out]
  warnings.warn(
/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/aeppl/joint_logprob.py:130: UserWarning: Found a random variable that was neither among the observations nor the conditioned variables: [normal_rv{0, (0, 0), floatX, False}.0, normal_rv{0, (0, 0), floatX, False}.out]
  warnings.warn(
/Users/marcoghezzi/PycharmProjects/pythonProject/ven

Sampling 2 chains for 2_000 tune and 1_000 draw iterations (4_000 + 2_000 draws total) took 198 seconds.
There were 75 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8193, but should be close to 0.9. Try to increase the number of tuning steps.
There were 344 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.4093, but should be close to 0.9. Try to increase the number of tuning steps.
/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/aeppl/joint_logprob.py:130: UserWarning: Found a random variable that was neither among the observations nor the conditioned variables: [normal_rv{0, (0, 0), floatX, False}.0, normal_rv{0, (0, 0), floatX, False}.out]
  warnings.warn(
/Users/marcoghezzi/PycharmProjects/pythonProject/venv/lib/python3.9/site-packages/aeppl/joint_logprob.py:130: UserWarning: Found a

In [ ]:
rolling.free_RVs